# LAB3
## FI-31 Burma Sofia

In [1]:

from spyre import server

import pandas as pd
import os
import urllib3
import re
import matplotlib.pyplot as plt

import __main__ as main
main.__file__ = "main_file"


In [2]:
def reading_text_into_the_frame(directory):
    
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    all_df = []
    fnames = os.listdir(directory)
   
    for fname in fnames:
        
        if fname.startswith("NOAA") and fname.endswith(".csv"):
            
            file_path = os.path.join(directory, fname)
            df = pd.read_csv(file_path, header=1, names=headers)
            df =df.drop(columns=['empty']).dropna()
            df = df.drop(df.loc[df['VHI'] == -1].index)
            
            i = int(re.search(r'\d+', fname).group())
            df['area'] = i
            
            df.loc[0, 'Year'] = df.loc[0,'Year'][9:]
            
            df['Year'] = df['Year'].astype(int)
            
            all_df.append(df)
            
    final_df = pd.concat(all_df, ignore_index=True)
 
  
    final_df['Year'] = pd.to_numeric(final_df['Year'], errors='coerce')
    final_df['area'] = pd.to_numeric(final_df['area'], errors='coerce')
    
    return(final_df)


In [3]:
directory = "../lab2/Data"
df = reading_text_into_the_frame(directory)

## Створення веб-додатоку 
  - обрати часовий ряд VCI, TCI, VHI для набору даних із лабораторної
роботи 2 (dropdown список);
  - Вибрати область, для якої буде виконуватись аналіз (dropdown список);
  - Зазначити інтервал тижнів, за які відбираються дані;
  - Зазначити інтервал років та продумати відображення на графіку обраних
даних (часові ряди задіапазон років, що обмежені інтервалом тижнів)
  - Створити кілька вкладок для відображення таблиці із даними на графіку
ходу індексів;
  - Інші завдання на прохання викладача.

In [ ]:
options = [ {"label": "Cherkasy", "value": 1},
                                  {"label": "Chernihiv", "value": 2},
                                  {"label": "Chernivtsi", "value": 3},
                                  {"label": "Crimea", "value": 4},
                                  {"label": "Dnipropetrovs'k", "value": 5},
                                  {"label": "Donets'k", "value": 6},
                                  {"label": "Ivano-Frankivs'k", "value": 7},
                                  {"label": "Kharkiv", "value": 8},
                                  {"label": "Kherson", "value": 9},
                                  {"label": "Khmel'nyts'kyy", "value": 10},
                                  {"label": "Kiev", "value": 11},
                                  {"label": "Kiev City", "value": 12},
                                  {"label": "Kirovohrad", "value": 13},
                                  {"label": "Luhans'k", "value": 14},
                                  {"label": "L'viv", "value": 15},
                                  {"label": "Mykolayiv", "value": 16},
                                  {"label": "Odessa", "value": 17},
                                  {"label": "Poltava", "value": 18},
                                  {"label": "Cherkasy", "value": 19},
                                  {"label": "Rivne", "value": 20},
                                  {"label": "Sumy", "value": 21},
                                  {"label": "Ternopil'", "value": 22},
                                  {"label": "Transcarpathia", "value": 23},
                                  {"label": "Vinnytsya", "value": 24},
                                  {"label": "Volyn", "value": 25},
                                  {"label": "Zaporizhzhya", "value": 26},
                                  {"label": "Zhytomyr", "value": 27}]

class SimpleApp(server.App):
    title = "NOAA data visuslization"

            # Oбрати часовий ряд VCI, TCI, VHI для набору даних із лабораторної роботи 2 (dropdown список)
    inputs = [
            { "type": 'dropdown',
                    "label": 'NOAA data dropdown',
                    "options" : [ {"label": "VCI", "value":"VCI"},
                                  {"label": "TCI", "value":"TCI"},
                                  {"label": "VHI", "value":"VHI"}],
                    "key": 'ticker',
                    'value': 'VHI',
                    "action_id": "update_data"},
            
            # Вибрати область, для якої буде виконуватись аналіз (dropdown список)
            {"type": 'dropdown',
                    "label": 'Select a region',

                    "options" : [ {"label": i["label"], "value": i["value"]} for i in options],
                    "key": 'area',
                    'value': 1,
                    "action_id": "update_data"},
            
            # Зазначити інтервал років та продумати відображення на графіку обраних даних
                 
            { "type":'text',
                    "label": 'The beginning of the interval (1982-2024)',
                    "key" :  'year_1' ,  
                    "value" :  1985 ,
                    "action_id"  :  "update_data" , 
                    },
             { "type":'text',
                    "label": 'The end of the interval (1982-2024)',
                    "key" :  'year_2' ,  
                    "value" :  2020 ,
                    "action_id" : "update_data", 
                    },
    
         
            #   Зазначити інтервал тижнів, за які відбираються дані
              { "type":'text',
                    "label": 'Interval of weeks',
                    "key" :  'range_week' ,  
                    "value" :  "1-3" ,
                    "action_id"  :  "update_data" , 
                    }]
            

    controls = [{  "type" : "hidden",
                    "id" : "update_data"}]
 
    #  Створити кілька вкладок для відображення таблиці із даними на графіку ходу індексів
    tabs = ["Plot", "Table"]

    outputs = [{ "type" : "plot",
                    "id" : "plot",
                    "control_id" : "update_data",
                    "tab" : "Plot"},
               
                { "type" : "table",
                    "id" : "table_id",
                    "control_id" : "update_data",
                    "tab" : "Table",
                    "on_page_load" : True }]

    def getData(self, params):
        
        ticker = params['ticker']
        area = int(params['area'])
        year_1, year_2 = int(params['year_1']), int((params['year_2']))
        week = (params['range_week'].split('-'))
        week_1, week_2 = int(week[0]), int(week[1])
        final_df = df.loc[ (df['area'] == area) &
                           (df['Week'].between(week_1, week_2)) &
                           (df['Year'].between(year_1, year_2)) ,
                            ['Year', 'Week', ticker] ] 
        return final_df

    def getPlot(self, params):
        
        label = "0"
        for dict in options:
                   
                if str(dict["value"])==str(params['area']): 
                      label = dict["label"]  
                      break
        
     
        
        df = self.getData(params).set_index(['Year','Week'])
        plt_obj = df.plot( title = params['ticker'] + " plot for "+ label , xlabel="Year, Week")
        
        return plt_obj.get_figure()


In [ ]:
app = SimpleApp()
app.launch()

[25/Nov/2024:17:34:11] ENGINE Listening for SIGTERM.
INFO:cherrypy.error:[25/Nov/2024:17:34:11] ENGINE Listening for SIGTERM.
[25/Nov/2024:17:34:11] ENGINE Bus STARTING
INFO:cherrypy.error:[25/Nov/2024:17:34:11] ENGINE Bus STARTING
[25/Nov/2024:17:34:11] ENGINE Set handler for console events.
INFO:cherrypy.error:[25/Nov/2024:17:34:11] ENGINE Set handler for console events.
CherryPy Checker:
The Application mounted at '' has an empty config.

[25/Nov/2024:17:34:11] ENGINE Started monitor thread 'Autoreloader'.
INFO:cherrypy.error:[25/Nov/2024:17:34:11] ENGINE Started monitor thread 'Autoreloader'.
[25/Nov/2024:17:34:12] ENGINE Serving on http://127.0.0.1:8080
INFO:cherrypy.error:[25/Nov/2024:17:34:12] ENGINE Serving on http://127.0.0.1:8080
[25/Nov/2024:17:34:12] ENGINE Bus STARTED
INFO:cherrypy.error:[25/Nov/2024:17:34:12] ENGINE Bus STARTED


127.0.0.1 - - [25/Nov/2024:17:34:21] "GET / HTTP/1.1" 200 442787 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0"


INFO:cherrypy.access.2380177277072:127.0.0.1 - - [25/Nov/2024:17:34:21] "GET / HTTP/1.1" 200 442787 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0"


1 1 Cherkasy
Cherkasy
127.0.0.1 - - [25/Nov/2024:17:34:21] "GET /spinning_wheel HTTP/1.1" 200 2663 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0"


INFO:cherrypy.access.2380177277072:127.0.0.1 - - [25/Nov/2024:17:34:21] "GET /spinning_wheel HTTP/1.1" 200 2663 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0"


127.0.0.1 - - [25/Nov/2024:17:34:21] "GET /table?ticker=VHI&area=1&year_1=1985&year_2=2020&range_week=1-3&output_id=table_id& HTTP/1.1" 200 8209 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0"


INFO:cherrypy.access.2380177277072:127.0.0.1 - - [25/Nov/2024:17:34:21] "GET /table?ticker=VHI&area=1&year_1=1985&year_2=2020&range_week=1-3&output_id=table_id& HTTP/1.1" 200 8209 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0"
ERROR:root:Error: getPlot method must return an pyplot figure or matplotlib Axes object


127.0.0.1 - - [25/Nov/2024:17:34:22] "GET /plot?ticker=VHI&area=1&year_1=1985&year_2=2020&range_week=1-3&output_id=plot& HTTP/1.1" 200 37661 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0"


INFO:cherrypy.access.2380177277072:127.0.0.1 - - [25/Nov/2024:17:34:22] "GET /plot?ticker=VHI&area=1&year_1=1985&year_2=2020&range_week=1-3&output_id=plot& HTTP/1.1" 200 37661 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0"
